# Projeto de Detecção de Dedos com a Webcam
Este projeto usa a biblioteca **MediaPipe** para detectar as mãos e contar a quantidade de dedos levantados em tempo real, usando a webcam do computador.

Você verá na tela a imagem da webcam com a contagem de dedos exibida.

In [2]:
!pip install mediapipe

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/51.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/51.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/51.0 MB ? eta -:--:--
    --------------------------------------- 1.0/51.0 MB 2.8 MB/s eta 0:00:18
   - -------------------------------------- 1.6/51.0 MB 2.6 MB/s eta 0:00:19
   - -------------------------------------- 1.6/51.0 MB 2.6 MB/s eta 0:00:19
   -- ------------------------------------- 2.6/51.0 MB 3.0 MB/s eta 0:00:17
   -- ------------------------------------- 2.6/51.0 MB 3.0 MB/s eta 0:00:17
   --- ------------------------------------ 3.9/51.0 MB 2.8 MB/s eta 0:00:18
   --- -----------------------

In [3]:
import cv2
import mediapipe as mp
import time

In [4]:
# Inicializa o módulo de detecção de mãos do MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Configurações do detector de mãos
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.5)

# Lista com os índices das pontas dos dedos
finger_tips_ids = [4, 8, 12, 16, 20]  # polegar, indicador, médio, anelar, mínimo

# Inicia a webcam
cap = cv2.VideoCapture(0)
pTime = 0

while True:
    success, frame = cap.read()
    if not success:
        break

    # Inverte o frame (espelhado) e converte de BGR para RGB
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    count = 0

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lm_list = []
            for id, lm in enumerate(handLms.landmark):
                h, w, _ = frame.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lm_list.append((id, cx, cy))

            # Contagem de dedos
            if lm_list:
                # Polegar (verifica se está à esquerda ou direita do ponto anterior)
                if lm_list[4][1] > lm_list[3][1]:
                    count += 1

                # Demais dedos (verifica se a ponta está acima da articulação)
                for tip_id in finger_tips_ids[1:]:
                    if lm_list[tip_id][2] < lm_list[tip_id - 2][2]:
                        count += 1

            mp_drawing.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)

    # Exibe a contagem de dedos
    cv2.rectangle(frame, (0, 0), (150, 80), (0, 0, 0), -1)
    cv2.putText(frame, f'Qtde de Dedos: {count}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX,
                1.8, (255, 255, 255), 2)

    # Exibe FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime) if (cTime - pTime) != 0 else 0
    pTime = cTime
    cv2.putText(frame, f'FPS: {int(fps)}', (450, 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.8, (0, 255, 0), 2)

    # Mostra o resultado
    cv2.imshow("Detecção de Dedos", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera recursos
cap.release()
cv2.destroyAllWindows()